In [1]:
import os
import json
import random
import numpy as np
import pandas as pd
import math
import torch
import gpytorch
import sys
import re
import statistics
from matplotlib import pyplot as plt
from scipy.stats import gaussian_kde
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from Bio.PDB import *

import Kidera

In [2]:
df = pd.read_csv('data/thermomutdb_v1.3.csv')
df = df.dropna(subset=['ID', 'TEMP', 'pH', 'PDB_wild', 'MUTATION_pdb', 'MUTATED_CHAIN', 'ddG'])
df = df[df['PDB_wild'] != '-'].sort_values(['PDB_wild']).reset_index(drop=True)
kidera_table = pd.read_csv('Kidera/kidera.csv', header=None).set_index(0)
ndatas = df.shape[0]

symbol_lookup = { 'ALA': 'A', 'ARG': 'R',
                  'ASN': 'N', 'ASP': 'D',
                  'CYS': 'C', 'GLN': 'Q',
                  'GLU': 'E', 'GLY': 'G',
                  'HIS': 'H', 'ILE': 'I',
                  'LEU': 'L', 'LYS': 'K',
                  'MET': 'M', 'PHE': 'F',
                  'PRO': 'P', 'SER': 'S',
                  'THR': 'T', 'TRP': 'W',
                  'TYR': 'Y', 'VAL': 'V' }
pdbl = PDBList()
pdb_parser = MMCIFParser(QUIET=True)

len(list(df['PDB_wild'].unique()))

337

In [3]:
def get_property(item):
    print(type(item))
    
    for key, value in item.__dict__.items():
        print(key, ':', value)
        
    for x in dir(item):
        print(x, ':', type(eval("item."+x)))
        
def get_C_vector(resi):
    if 'CB' in resi:
        return resi['CB'].get_vector()
    elif 'CA' in resi:
        return resi['CA'].get_vector()

In [4]:
%%time

structure = None
pre_pdb = None
count = 0
result_df = pd.DataFrame({'ID': [],        'TEMP': [],      'pH': [],       'PDB': [], 
                          'MUTATION': [],  'chain': [],     'ddG': [],      'A': [], 
                          'R': [],         'N': [],         'D': [],        'C': [],
                          'Q': [],         'E': [],         'G': [],        'H': [],
                          'I': [],         'L': [],         'K': [],        'M': [],
                          'F': [],         'P': [],         'S': [],        'T': [],
                          'W': [],         'Y': [],         'V': [],        'ALA': [], 
                          'ARG': [],       'ASN': [],       'ASP': [],      'CYS': [], 
                          'GLN': [],       'GLU': [],       'GLY': [],      'HIS': [], 
                          'ILE': [],       'LEU': [],       'LYS': [],      'MET': [], 
                          'PHE': [],       'PRO': [],       'SER': [],      'THR': [], 
                          'TRP': [],       'TYR': [],       'VAL': [],      'Kidera_0': [], 
                          'Kidera_1': [],  'Kidera_2': [],  'Kidera_3': [], 'Kidera_4': [], 
                          'Kidera_5': [],  'Kidera_6': [],  'Kidera_7': [], 'Kidera_8': [], 
                          'Kidera_9': []})
for index, row in df.iterrows():
    # print(row)
    
    is_invalid = False
    is_invalid |= len(row['MUTATION_pdb'].split(',')) != 1
    if is_invalid:
        continue
    
    try:
        res_id = int(row['MUTATION_pdb'][1:-1])
        from_res = row['MUTATION_pdb'][0]
        to_res = row['MUTATION_pdb'][-1]
    except ValueError as e:
        print('ValueError at int(row["MUTATION_pdb"][1:-1]): {} {}'.format(index, row["MUTATION_pdb"]))
        continue
        
    pdb_id = row['PDB_wild'].lower()
    chain_id = row['MUTATED_CHAIN']
        
    if pre_pdb != pdb_id:
        if not os.path.exists('pdb_files/{}.cif'.format(pdb_id)):
            pdbl.retrieve_pdb_file(pdb_id, pdir='pdb_files/')
        structure = pdb_parser.get_structure(pdb_id, 'pdb_files/{}.cif'.format(pdb_id))
        pre_pdb = pdb_id
        
    try:
        chain = structure[0][chain_id]
        target_res = chain[res_id]
    except KeyError as e:
        print('KeyError at chain = structure[0][chain_id]: {}'.format(index))
        continue
        
    if symbol_lookup[target_res.resname] != from_res:
        continue
        
    target_atom = get_C_vector(target_res)
    kidera_vec = np.zeros(10)
    around_res = {'ALA': 0,   'ARG': 0,   'ASN': 0,   'ASP': 0,   'CYS': 0,   'GLN': 0,
                  'GLU': 0,   'GLY': 0,   'HIS': 0,   'ILE': 0,   'LEU': 0,   'LYS': 0,   'MET': 0, 
                  'PHE': 0,   'PRO': 0,   'SER': 0,   'THR': 0,   'TRP': 0,   'TYR': 0,   'VAL': 0}
    
    for res in chain.get_list():
        if res.id == target_res.id:
            continue
        if res.resname not in symbol_lookup:
            continue
        
        atom = get_C_vector(res)
        if atom is None:
            continue
        atom -= target_atom
        dist = np.linalg.norm(atom)
        if dist > 9:
            continue

        around_res[res.resname] += 1
        kidera_vec += np.array(kidera_table.loc[res.resname] / dist)
        
    # print(kidera_vec)
    count += 1
    series = pd.Series({'ID': row['ID'],     'TEMP': row['TEMP'],      'pH': row['pH'],    'PDB': row['PDB_wild'], 
                       'MUTATION': row['MUTATION_pdb'],  'chain': row['MUTATED_CHAIN'],     'ddG': row['ddG'], 
                       'A': 0,  'R': 0,  'N': 0,  'D': 0,  'C': 0,  'Q': 0,  'E': 0,  'G': 0,  'H': 0,  'I': 0,  
                       'L': 0,  'K': 0,  'M': 0,  'F': 0,  'P': 0,  'S': 0,  'T': 0,  'W': 0,  'Y': 0,  'V': 0,  
                       'ALA': around_res['ALA'],   'ARG': around_res['ARG'],   'ASN': around_res['ASN'],   'ASP': around_res['ASP'],   
                       'CYS': around_res['CYS'],   'GLN': around_res['GLN'],   'GLU': around_res['GLU'],   'GLY': around_res['GLY'],   
                       'HIS': around_res['HIS'],   'ILE': around_res['ILE'],   'LEU': around_res['LEU'],   'LYS': around_res['LYS'],   
                       'MET': around_res['MET'],   'PHE': around_res['PHE'],   'PRO': around_res['PRO'],   'SER': around_res['SER'],   
                       'THR': around_res['THR'],   'TRP': around_res['TRP'],   'TYR': around_res['TYR'],   'VAL': around_res['VAL'], 
                       'Kidera_0': kidera_vec[0],   'Kidera_1': kidera_vec[1],  'Kidera_2': kidera_vec[2],
                       'Kidera_3': kidera_vec[3],   'Kidera_4': kidera_vec[4],  'Kidera_5': kidera_vec[5], 
                       'Kidera_6': kidera_vec[6],   'Kidera_7': kidera_vec[7],  'Kidera_8': kidera_vec[8], 
                       'Kidera_9': kidera_vec[9]})
    series[from_res] = -1
    series[to_res] = 1
    
    result_df = result_df.append(series, ignore_index=True)

    # break

print('count is : {}'.format(count))
result_df

count is : 7989
CPU times: user 3min 26s, sys: 4.39 s, total: 3min 30s
Wall time: 3min 30s


,ID,TEMP,pH,PDB,MUTATION,chain,ddG,A,R,N,...,Kidera_0,Kidera_1,Kidera_2,Kidera_3,Kidera_4,Kidera_5,Kidera_6,Kidera_7,Kidera_8,Kidera_9
0,13111.0,298.15,7.5,12CA,G145R,A,-0.360000,0.0,1.0,0.0,...,-1.585343,0.492594,0.661326,-1.242804,-0.365329,-1.352580,-0.629848,0.157755,0.552728,0.610638
1,13110.0,298.15,7.5,12CA,H94Y,A,-1.170000,0.0,0.0,0.0,...,-0.820398,0.015932,-0.396325,-0.651443,0.447063,-0.789494,-0.089048,0.811497,1.706994,-0.515717
2,8625.0,329.83,6.5,1A0F,S11A,A,-1.800000,1.0,0.0,0.0,...,0.060134,-1.311882,-0.178009,-0.744594,0.292331,-1.418528,0.248968,-0.555226,0.178872,0.672534
3,4405.0,303.15,7.0,1A23,P151A,A,-3.019013,1.0,0.0,0.0,...,-0.498620,-0.490843,-0.182037,-1.393750,0.303348,0.058314,0.246750,0.264783,1.079031,0.366582
4,3661.0,303.15,6.3,1A23,H32S,A,-5.200000,0.0,0.0,0.0,...,0.829904,-0.040843,0.022546,-1.418299,0.406216,0.076213,0.958939,-1.097800,1.455064,-0.256101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7984,11815.0,278.15,7.0,5TR5,I44A,A,-1.148325,1.0,0.0,0.0,...,-0.840592,-0.307056,0.772631,0.042227,0.181924,-0.699600,-0.292672,0.762266,0.122844,0.704220
7985,11320.0,298.15,8.0,5VP3,R39K,A,0.413480,0.0,-1.0,0.0,...,0.195955,0.991407,0.012329,-0.041153,-0.266360,-0.789183,0.482266,0.269042,0.462153,0.195984
7986,11321.0,298.15,8.0,5VP3,S128G,A,-0.377629,0.0,0.0,0.0,...,-0.884315,0.336544,0.560522,-1.068174,-1.068412,-1.967318,-0.015810,0.189277,0.834460,0.325178
7987,11322.0,298.15,8.0,5VP3,V183T,A,0.353728,0.0,0.0,0.0,...,-0.194854,-0.126772,-0.116993,0.058190,-0.796814,-0.899134,-0.464923,-0.519305,0.777412,0.083511


In [5]:
result_df['ID'] = result_df['ID'].astype('int')
result_df.to_csv('data/DDG_Dataset_dist_Kidera.csv', index=False)